## WebページとWebAPIの違いを体験しよう

例題：郵便番号検索
- 郵便番号を元に住所を出力する。
- Webページ：郵便局の郵便番号検索サービス(https://www.post.japanpost.jpjp/cgi-zip/zipcodd.php)
- WebAPI：郵便番号データ配信サービス(https://zipcloud.ibsnet.co.jp/api/search)

## Webページのスクレイピング

In [1]:
from requests import get
from bs4 import BeautifulSoup

ページのダウンロード
 - `<Response [200]>`ならば成功！

In [2]:
page = ' http://www.post.japanpost.jp/cgi-zip/zipcode.php'
params = {'zip':'3590026'}
r = get(page, params=params)
r

<Response [200]>

`r.content`がバイト列であることを確認

In [3]:
type(r.content)

bytes

ブラウザで http://www.post.japanpost.jp/cgi-zip/zipcode.php?zip=？？？ を開き、ソースコード(HTML)内のどこに郵便番号と住所が存在するかを調べる。
- 欲しい情報は5箇所（郵便番号、都道府県、市区町村、町域、町域のフリガナ）

In [8]:
soup = BeautifulSoup(r.content, 'html.parser')
tag = 'td'
#val = 'data'
#tables = soup.find_all(tag, class_=val)
tables = soup.find_all(tag)
tables

[<td class="data">〒<small>359-0026</small></td>,
 <td class="data"><small>埼玉県</small></td>,
 <td class="data"><small>所沢市</small></td>,
 <td>
 <div class="data">
 <p><small><a class="line" href="zipcode.php?pref=11&amp;city=1112080&amp;id=39652&amp;merge=">牛沼</a></small></p>
 <span class="comment_zipsearch"><small>ウシヌマ</small></span>
 </div>
 </td>,
 <td class="data comment">
 <small> </small><br/>
 </td>,
 <td class="lb-dot comment">
 <div class="data">
 <p><small> </small></p>
 </div>
 </td>,
 <td class="lb-dot">
 <div class="data comment">
 <p><small> </small></p>
 <p class="sp-t5"><small> </small></p>
 </div>
 </td>,
 <td class="data comment arrange-c">
 <small> </small><br/>
 </td>]

スクレイピング
1. HTMLのパーズ(`BeautifulSoup`)
1. 5個の情報を抽出

In [13]:
soup = BeautifulSoup(r.content)
tag = 'td'
s = soup.find_all(tag)

zipcode = s[0].get_text() # 郵便番号をzipcodeに代入
prefecture = s[1].get_text() # prefectureをzipcodeに代入
city = s[2].get_text() # cityをzipcodeに代入
town = s[3].find('a').get_text() # townをzipcodeに代入
kana = s[3].find('span').get_text() # Kanaをzipcodeに代入
'''
適切な文字列をzipcode, prefecture, city, town, kanaに代入するコード
'''
print(f'郵便番号：{zipcode}')
print(f'都道府県：{prefecture}')
print(f'市区町村：{city}')
print(f'町域：{town}')
print(f'フリガナ：{kana}')

郵便番号：〒359-0026
都道府県：埼玉県
市区町村：所沢市
町域：牛沼
フリガナ：ウシヌマ


## WebAPI

In [5]:
from requests import get
import json

ページのダウンロード
 - `<Response [200]>`ならば成功！

In [6]:
page = 'https://zipcloud.ibsnet.co.jp/api/search'
params = {'zipcode':'3590026'}
r = get(page, params=params)
r

<Response [200]>

`r.content`は、Webページのときと同様に、バイト列であることを確認

In [7]:
type(r.content)

bytes

`r.content`を文字列（JSONコード）に変換して表示

In [8]:
print(r.content.decode())

{
	"message": null,
	"results": [
		{
			"address1": "埼玉県",
			"address2": "所沢市",
			"address3": "牛沼",
			"kana1": "ｻｲﾀﾏｹﾝ",
			"kana2": "ﾄｺﾛｻﾞﾜｼ",
			"kana3": "ｳｼﾇﾏ",
			"prefcode": "11",
			"zipcode": "3590026"
		}
	],
	"status": 200
}


JSONをパーズしてPythonのデータに変換し、型を確認する

In [9]:
data = r.json()
print(f'型：{type(data)}')
data

型：<class 'dict'>


{'message': None,
 'results': [{'address1': '埼玉県',
   'address2': '所沢市',
   'address3': '牛沼',
   'kana1': 'ｻｲﾀﾏｹﾝ',
   'kana2': 'ﾄｺﾛｻﾞﾜｼ',
   'kana3': 'ｳｼﾇﾏ',
   'prefcode': '11',
   'zipcode': '3590026'}],
 'status': 200}

`data`から情報を抽出する
- 郵便局のページでは表示されていない都道府県・市区町村のフリガナも取得できる。

In [10]:
zipcode = data['results'][0]['zipcode'] # 郵便番号をzipcodeに代入
prefecture = data['results'][0]['address1'] # prefectureをzipcodeに代入
city = data['results'][0]['address2'] # cityをzipcodeに代入
town = data['results'][0]['address3'] # townをzipcodeに代入
prefectureKana = data['results'][0]['kana1'] # prefectureKanaをzipcodeに代入
cityKana = data['results'][0]['kana2'] # cityKanaをzipcodeに代入
townKana = data['results'][0]['kana3'] # townKanaをzipcodeに代入
'''
適切な文字列をprefecture, city, town, prefectureKana, cityKana, townKanaに代入するコード
'''
print(f'郵便番号：{zipcode}')
print(f'都道府県：{prefecture}')
print(f'市区町村：{city}')
print(f'町域：{town}')
print(f'フリガナ：{prefectureKana}/{cityKana}/{townKana}')

郵便番号：3590026
都道府県：埼玉県
市区町村：所沢市
町域：牛沼
フリガナ：ｻｲﾀﾏｹﾝ/ﾄｺﾛｻﾞﾜｼ/ｳｼﾇﾏ
